# DIC Crack Inspector

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from io import BytesIO

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
from bmcs_shear.dic_crack import sz_tests_series_2023 as ts
import numpy as np
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
tests = [ts.B1_TV1, ts.B1_TV2, ts.B6_TV1, ts.B6_TV2, ts.B7_TV1, ts.B7_TV2, 
         ts.B8_TV1, ts.B8_TV2, ts.B9_TV1, ts.B9_TV2, ts.B10_TV1, ts.B10_TV2]

In [ ]:
def fig_test_subplots():
    fig = plt.figure(figsize=(10, 10))
    fig.canvas.header_visible = False
    grid = gridspec.GridSpec(3, 3, width_ratios=[1, 1, 1], height_ratios=[0.4, 0.4, 0.7])
    ax_list = [fig.add_subplot(grid[0, :]),     
               fig.add_subplot(grid[1, :2]),
               fig.add_subplot(grid[1, 2])] + \
        [fig.add_subplot(grid[2, i]) for i in range(3)]
    return fig, ax_list

In [ ]:
active_tests = tests #  [tests[1]]
dcl_tests = [ts.new_dcl(test) for test in active_tests]



In [ ]:
def plot_crack_detection_field(dcl, ax, t_detect=None):
    if t_detect is None:
        t_detect = dcl.t_detect
    dcl.dsf.dic_grid.t = t_detect
    xx_MN, yy_MN, cd_field_irn_MN = dcl.dsf.omega_irn_1_MN
    cd_field_irn_MN[cd_field_irn_MN<0.1] = 0
    contour_levels = np.array([0.15, 0.35, 0.65, 0.85, 1.05], dtype=np.float_)
    cs = ax.contourf(xx_MN, yy_MN, cd_field_irn_MN, contour_levels,
                            #cmap=cm.GnBu,
                            #cmap=cm.coolwarm,
                            antialiased=False)
    ax.set_aspect("equal")

In [ ]:
# dcl = dcl_tests[0]
# fig, (ax) = plt.subplots(1,1, figsize=(10,3))
# fig.canvas.header_visible=False
# plot_crack_detection_field(dcl, ax)
# dcl.dsf.dic_grid.sz_bd.plot_sz_bd(ax)
# dcl.plot_primary_cracks(ax_cracks=ax)
# ax.set_title(f'Test {dcl.dir_name}')
# ax.set_aspect("equal")

In [ ]:
n_tests = len(dcl_tests)
figs = []
for dcl in dcl_tests:
    fig, (ax, ax_cracks, ax_ld, ax1, ax2, ax3) = fig_test_subplots()
    # dcl.bd.plot_sz_bd(ax)
    plot_crack_detection_field(dcl, ax)
    dcl.dsf.dic_grid.sz_bd.plot_sz_bd(ax)
    # dcl.plot_primary_cracks(ax_cracks=ax)
    ax.set_title(f'Test specimen {dcl.dir_name}: FE-DIC damage at $F_{{\max}}$',
        loc='left')
    ax.title.set_position([.0, 0.95])
    ax.set_aspect("equal")

    dcl.dsf.plot_crack_detection_field(ax_cracks, fig)
    dcl.plot_primary_cracks(ax_cracks=ax_cracks)
    ax_cracks.set_aspect("equal")
    ax_cracks.set_title(f'Cracks at detection state $t$={dcl.t_detect}$F_{{\max}}$ extended to ultimate state', loc='left')
    ax.title.set_position([0,1])
    ax_cracks.set_aspect("equal")

    dcl.dsf.dic_grid.dic_inp.plot_load_deflection(ax_ld)
    ax_ld.set_title('Load-deflection')

    M_Ct, phi_Ct = np.einsum('CMt->MCt', dcl.M_phi_Ct)
    phi_Ct = np.rad2deg(phi_Ct)

    for i, (cr, M_t, phi_t) in enumerate(zip(dcl.cracks, M_Ct, phi_Ct), start=1):
        F_T = cr.dic_grid.dic_inp.F_T
        T_range = F_T / F_T[-1]
        T_range[T_range<0] = 0
        u_Ta = np.max(cr.get_u_crc_Ka(T_range, cr.X_crc_1_Ka[:5,:]), axis=1)
        ax1.plot(u_Ta[:, 0], F_T, label=f'crack {cr.name}', color=cr.color)
        ax2.plot(u_Ta[:, 1], F_T, label=f'crack {i}', color=cr.color)
        ax3.plot(phi_t, M_t, label=f'crack {cr.name}', color=cr.color)
    ax1.set_ylabel(r'$F$/kN')
    ax1.set_xlabel(r'$w$/mm')
    ax1.set_title('Load vs. crack opening')
    ax1.grid()

    ax2.set_ylabel(r'$F$/kN')
    ax2.set_xlabel(r'$s$/mm')
    ax2.set_title('Load vs. slip')
    ax2.grid()
    ax2.legend()

    ax3.set_ylabel(r'$M$/kNm')
    ax3.set_xlabel(r'$\phi$/deg')
    ax3.set_title('Bending moment vs. tooth rotation')
    ax3.grid()

    fig.subplots_adjust(wspace=0.3, hspace=0.4)
    figs.append(fig)

\begin{equation}
M = \frac{F}{2} (L - X^{cor})
\end{equation}

In [ ]:
import os
# Function to create a PDF report with figures and text
def create_report(filename, figures, texts):
    c = canvas.Canvas(filename, pagesize=A4)
    
    # Add figures to the report
    for i, fig in enumerate(figures):
        img_data = BytesIO()
        fig.savefig(img_data, format='png')
        img_data.seek(0)
        
        # Save the image data to a temporary PNG file
        img_path = f"temp_image_{i}.png"
        with open(img_path, 'wb') as img_file:
            img_file.write(img_data.read())
        
        # Draw the image from the file path
        c.drawImage(img_path, 0, 200, width=600, height=400)
        
        # Delete the temporary image file
        os.remove(img_path)
        c.showPage()  # Move to the next page
    
    c.save()

# Example figures and text
figures = [plt.figure(), plt.figure()]  # Add your matplotlib figures here
texts = [
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
    "This is the report text.",
]

# Create the PDF report
create_report("report.pdf", figs, texts)

In [ ]:
M_tC, phi_tC = np.einsum('CMt->MtC', dsl.M_phi_Ct)
M_tC.shape
